## Q17.1

我们可以参考Q16.2的方法来进行计算。

In [1]:
class Node:
    """A binary tree node.

    Args:
        stock: stock price at this node
        u: up rate
        d: down rate
        er: risk-free rate of interest
    """
    def __init__(self, stock, u, d, er):
        self.stock  = stock
        self.u      = u
        self.d      = d
        self.er     = er

        self._q     = None      ## risk-neutral probability
        self._price = None      ## asset price

        self.left   = None      ## left child node (the up branch)
        self.right  = None      ## right child node (the down branch)
    
    def is_leaf(self):
        return self.left == None and self.right == None
    
    @property
    def q(self):
        if self._q is None:
            self._q = (self.er - self.d) / (self.u - self.d)
        
        return self._q
    
    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, value):
        self._price = value
    
    @property
    def up_branch(self):
        return self.left
    
    @up_branch.setter
    def up_branch(self, node):
        self.left = node
    
    @property
    def down_branch(self):
        return self.right
    
    @down_branch.setter
    def down_branch(self, node):
        self.right = node

In [2]:
def create_layer(parents):
    """Create a layer of nodes with given parents.
    """

    nodes = []

    up_node = None

    for pa in parents:
        ## retrieve information from pa
        stock = pa.stock
        u     = pa.u
        d     = pa.d
        er    = pa.er

        ## initialize up branch
        if up_node is None:
            up_node = Node(stock * u, u, d, er)
            nodes.append(up_node)
        
        ## create down branch from pa
        down_node = Node(stock * d, u, d, er)

        ## link pa with up_node and down_node
        pa.up_branch   = up_node
        pa.down_branch = down_node

        ## switch down_node to up_node
        up_node = down_node

        ## add to nodes list
        nodes.append(up_node)

    return nodes

def create_pyramid(n_layer, stock, u, d, er):
    """Create a pyramid (binary tree) of nodes, and return its root.
    """

    ## initialize the root
    root = Node(stock, u, d, er)

    nodes = [root]

    for i in range(n_layer-1):
        nodes = create_layer(nodes)

    return root

In [3]:
## option function generator
def OptionGenerator(option_func):
    """An option generator that creates the option price function.
    """

    def AmericanOption(node):
        """American Option price function
        """

        ## return option price if we have found it
        if node.price is not None:
            return node.price

        ## otherwise solve it recursively
        if node.is_leaf():
            node.price = option_func(node)
        else:
            q  = node.q
            er = node.er

            node.price = max(option_func(node), 
                             (q * AmericanOption(node.up_branch) + (1-q) * AmericanOption(node.down_branch)) / er)

        return node.price
    
    return AmericanOption

In [4]:
import math
from functools import partial

stock = 2.341           ## stock price today

sigma   = 17.7 / 100    ## price volatility
delta_t = 246           ## number of trading days
duration= 49            ## periods

## constants
u = math.exp(sigma * math.sqrt(1 / delta_t))
d = 1.0 / u
er = math.exp(0.00011)
K = 2.5

## option price function
def AmericanPut(S, K):
    return max(K - S, 0.)

Option = OptionGenerator(lambda node: partial(AmericanPut, K=K)(node.stock))

## build the pyramid
root = create_pyramid(duration+1, stock, u, d, er)

print(f"The option price today is: {Option(root):.6f}")

The option price today is: 0.173660


## Q17.2

(a) 这里重新设计二叉树数据结构和贷款的价值函数进行计算。

In [5]:
class BondNode:
    """A binary tree node for bond.

    Args:
        B: the rest bond at this node
        rs: risk-free rate of interest (this could change)
        rb: bond rate (this does not change)
        q: risk-neutral probability
    """
    def __init__(self, B, rs, rb, q):
        self.B      = B
        self.rs     = rs
        self.rb     = rb
        self.q      = q
        
        self.value  = None      ## bond value

        self.left   = None      ## left child node (the up branch)
        self.right  = None      ## right child node (the down branch)
    
    def is_leaf(self):
        return self.left == None and self.right == None
    
    @property
    def up_branch(self):
        return self.left
    
    @up_branch.setter
    def up_branch(self, node):
        self.left = node
    
    @property
    def down_branch(self):
        return self.right
    
    @down_branch.setter
    def down_branch(self, node):
        self.right = node

In [6]:
def create_layer(parents, repay, u, d):
    """Create a layer of nodes with given parents.
    """

    nodes = []

    up_node = None

    for pa in parents:
        ## retrieve information from pa
        B  = max(pa.B - repay, 0)
        rs = pa.rs
        rb = pa.rb
        q  = pa.q

        ## initialize up branch
        if up_node is None:
            up_node = BondNode(B, rs * u, rb, q)
            nodes.append(up_node)
        
        ## create down branch from pa
        down_node = BondNode(B, rs * d, rb, q)

        ## link pa with up_node and down_node
        pa.up_branch   = up_node
        pa.down_branch = down_node

        ## switch down_node to up_node
        up_node = down_node

        ## add to nodes list
        nodes.append(up_node)

    return nodes

def create_pyramid(n_layer, B, rs, rb, q, repay, u, d):
    """Create a pyramid (binary tree) of nodes, and return its root.
    """

    ## initialize the root
    root = BondNode(B, rs, rb, q)

    nodes = [root]

    for i in range(n_layer-1):
        nodes = create_layer(nodes, repay, u, d)

    return root

In [7]:
def BondValue(node):
    """Bond value function.
    """

    ## return bond value if we have found it
    if node.value is not None:
        return node.value
    
    ## otherwise solve it recursively
    if node.is_leaf():
        node.value = node.B
    else:
        q  = node.q
        rs = node.rs
        rb = node.rb

        up_branch   = (1 + rb) * node.B - node.up_branch.B + BondValue(node.up_branch)
        down_branch = (1 + rb) * node.B - node.down_branch.B + BondValue(node.down_branch)
        node.value  = (q * up_branch + (1-q) * down_branch) / (1 + rs)

    return node.value

In [8]:
B           = 100
rs          = 5 / 100
rb          = 5.5 / 100
q           = 0.5

repay       = 10
u           = math.exp(0.1)
d           = math.exp(-0.1)
duration    = 10

## build the pyramid
root = create_pyramid(duration+1, B, rs, rb, q, repay, u, d)

print(f"The bond value today is: {BondValue(root):.6f}.")

The bond value today is: 102.051622.


(b) 对于允许提前还款的情况，我们只需要更新价值函数即可。不过这里需要注意在根节点上无需比较是否应该提前还款。

In [9]:
def BondValue(node, root=True):
    """Bond value function.
    """

    ## return bond value if we have found it
    if node.value is not None:
        return node.value
    
    ## otherwise solve it recursively
    if node.is_leaf():
        node.value = node.B
    else:
        q  = node.q
        rs = node.rs
        rb = node.rb

        up_branch   = (1 + rb) * node.B - node.up_branch.B + BondValue(node.up_branch, False)
        down_branch = (1 + rb) * node.B - node.down_branch.B + BondValue(node.down_branch, False)

        if root:
            node.value = (q * up_branch + (1-q) * down_branch) / (1 + rs)
        else:
            node.value = min(node.B, (q * up_branch + (1-q) * down_branch) / (1 + rs))

    return node.value

In [10]:
## build the pyramid
root = create_pyramid(duration+1, B, rs, rb, q, repay, u, d)

print(f"The bond value today is: {BondValue(root, True):.6f}.")

The bond value today is: 99.972623.
